# Cloning the git

In [ ]:
import os

if "master" not in os.listdir("./"):
    !git clone https://github.com/YounesHB92/master.git
    !pip install -r ./master/requirements.txt
else:
    print("Already cloned!")

# Adding the path to the system path

In [ ]:
import sys

if "/content/master" not in sys.path:
    sys.path.append('/content/master')
else:
    print("Already added!")

# Downloading datasets

In [ ]:
if "datasets.tar.gz" not in os.listdir("./"):
    !gdown "https://drive.google.com/uc?id=1Gtw4q4pkMHWMYz5urew5C44X17oLE9e3"
    !tar -xvf datasets.tar.gz -C ./master
else:
    print("Already downloaded!")

# Cloning google drive

In [ ]:
if "drive" not in os.listdir("./"):
    from google.colab import drive
    drive.mount('/content/drive')
else:
    print("Already mounted!")

In [ ]:
import os

from src.utils import find_configs, load_env_variables

env = load_env_variables()
from src.datasets import Splitter, DatasetIterator, DatasetLoader
from src.training import Trainer, LossAndMetrics
from src.models import LoadModel
from datetime import datetime
import torch
import warnings
import yaml
import pytz
time_zone = pytz.timezone("Australia/Brisbane")

In [ ]:
warnings.filterwarnings("ignore")

print("Loading environment variables for:", env)

for dir_ in ["CHECKPOINTS_DIR", "LOGS_DIR"]: # make the output folders if they do not exist
    os.makedirs(os.getenv(dir_), exist_ok=True)
print("Output folders created.")

configs = find_configs()
print("Number of configurations found: ", len(configs.keys()))

for config_name in configs.keys():
    print(f"Configuration: {config_name}")

    config = configs[config_name]

    # saving the config file
    config_save_name = f"{config_name}_{datetime.now(time_zone).strftime('%Y-%m-%d_%H-%M-%S')}"
    with open(os.path.join(os.getenv("CHECKPOINTS_DIR"), config_save_name + ".yaml"), "w") as file:
        yaml.dump(config, file)

    splitter_ = Splitter(**config["splitter"])
    train_iterator_ = DatasetIterator(**config["datasets"]["train"]["iterator"])
    val_iterator_ = DatasetIterator(**config["datasets"]["val"]["iterator"])

    train_loader_ = DatasetLoader(
        dataset=train_iterator_,
        classes=splitter_.classes,
        **config["datasets"]["train"]["loader"]
    )
    train_loader = train_loader_.loader

    val_loader_ = DatasetLoader(
        dataset=val_iterator_,
        classes=splitter_.classes,
        **config["datasets"]["val"]["loader"]
    )
    val_loader = val_loader_.loader

    model_loader_ = LoadModel(
        model_name=config["model"]["model_name"],
        encoder_name=config["model"]["encoder_name"],
        encoder_weights=config["model"]["encoder_weights"],
        num_classes=len(splitter_.classes)
    )

    model = model_loader_.load_model()
    print("Model loaded successfully.")

    loss_and_metrics_ = LossAndMetrics(num_classes=len(splitter_.classes))
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    trainer_ = Trainer(
        model=model,
        optimizer=optimizer,
        loss_metrics=loss_and_metrics_,
        train_loader=train_loader,
        config_name=config_save_name,
        val_loader=val_loader,
        device="cuda",
        **config["trainer"]
    )

    trainer_.train()
